<a href="https://colab.research.google.com/github/am1235le/GY7720_Dissertation_209041686/blob/main/gee_data_access_run_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract GEE Data into Google Storage Data Bucket

Some sections of the following code that is used to access data from tfrecords was sourced and adapted from the code made available by Huot et al. (2022), which can be found under the following link:

https://github.com/google-research/google-research/tree/master/simulation_research/next_day_wildfire_spread

Huot F., Hu R. L., Goyal N., Sankar T., Ihme M. and Chen Y. F. 2022. Next Day Wildfire Spread: A Machine Learning Dataset to Predict Wildfire Spreading From Remote-Sensing Data. IEEE Transactions on Geoscience and Remote Sensing, vol. 60, pp. 1-13, 2022, Art no. 4412513, doi: 10.1109/TGRS.2022.3192974.

## Install and load libraries

In [ ]:
# install missing library
!pip install tensorflow-addons

In [ ]:
# load libraries
# base libraries
import re
from typing import Dict, List, Optional, Text, Tuple
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import random
import numpy as np
import pandas as pd
import seaborn as sns
import shutil, sys
import gzip
import zipfile
import os
from os import listdir
from os.path import isfile, isdir, join

# data analysis library
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")
from sklearn.datasets import make_blobs

# required for plotting
from matplotlib.colors import ListedColormap
from matplotlib.colors import ListedColormap, BoundaryNorm

# tensor flow libraries
import tensorflow as tf
from tensorflow import keras
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Reshape,
    Dropout,
    BatchNormalization,
    Conv1D,
    Conv2D,
    Conv2DTranspose,
    MaxPooling2D,
    SimpleRNN,
    LSTM,
    GRU,
)
from tensorflow.keras.losses import (
    Loss,
    BinaryFocalCrossentropy)
import tensorflow_addons as tfa

## Connect to Google Drive and Google Storage Bucket

In [ ]:
# connect to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# set working directory for data to be saved to
%cd /content/drive/My\ Drive/Colab\ Notebooks/Dissertation/data/

In [ ]:
# connect to project on Google Cloud containing Google storage bucket
from google.colab import auth
auth.authenticate_user()
project_id = 'project_name'
!gcloud config set project {project_id}
!gsutil ls

In [ ]:
# copy data from Google Cloud bucket folder to local Google Drive
bucket_name = 'bucket_name/folder_name'
!gsutil -m cp -r gs://{bucket_name} /content/drive/My\ Drive/Colab\ Notebooks/Dissertation/data/

In [ ]:
# list data/folders in working directory
! ls

## Define features to be extracted from tfrecords and used in model

In [10]:
# define list of features for extraction from tfrecord
# training features
INPUT_FEATURES = ['elevation',
                  'populationdensity',
                  'lulc', 
                  'pdsi',
                  'pr',
                  'erc',
                  'SPFH',
                  'WDIR',
                  'WIND',
                  'GUST',
                  'UGRD',
                  'VGRD',
                  'TMP',
                  'ndvid',
                  'ndii',
                  'nbr',
                  'dnbr',
                  'PrevFireMask'
                  ]
# target labels
OUTPUT_FEATURES = ['FireMask', ]

# Data statistics for each rescaling each variable
# the statistics are ordered in the form:
# (min_clip, max_clip, mean, std)
DATA_STATS = {
    # Elevation m
    'elevation': (-50, 6500, 1400, 280),
    # Population p/sqkm
    'populationdensity': (0, 810694, 7, 88),
    # Urban LULC
    'lulc': (0,1,0.5,0.5),
    # Palmer Drought Severity Index no units
    'pdsi': (-15, 15, -2, 0.6),
    # Precipitation mm, daily total
    'pr': (0, 100, 0.04, 0.06),
    # NFDRS fire danger index Energy release component
    'erc': (0.0, 131.85, 76.68, 2.25),
    # Specific humidity
    'SPFH': (0, 0.02, 0.01, 0.01),
    # Wind direction
    'WDIR': (0, 360, 212, 20),
    # Wind speed
    'WIND': (0, 42.46, 3.1, 0.2),
    # Wind gust
    'GUST': (0, 58.02, 10, 1),
    # Wind U
    'UGRD': (-32.93, 34.04, 0, 33),
    # Wind V
    'VGRD': (-28.44, 39.21, 5, 35),
    # Temperature
    'TMP': (-43.2, 43.73, 0, 43),
    # Normalised Difference Vegetation Index
    'ndvid': (-1, 1, 0, 1),
    # Normalised Difference Infrared Index
    'ndii': (-1, 1, 0, 1),
    # Normalised Burn Ratio
    'nbr': (-1, 1, 0, 1),
    # Difference Normalised Burn Ratio
    'dnbr': (-2, 2, 0, 2),
    # FireMasks.
    'PrevFireMask': (0, 1, 0.5, 0.5),
    'FireMask': (0, 1, 0.5, 0.5)
}

In [ ]:
# list defined date feature statistics
print(DATA_STATS)

## Define function to extract tfrecord data

In [12]:
# define functions to extract data from tfrecords
# sourced from Huot et al. (2022)

# function to randomly crop input and output images
"""Library of common functions used in deep learning neural networks.
"""
def random_crop_input_and_output_images(
    input_img: tf.Tensor,
    output_img: tf.Tensor,
    sample_size: int,
    num_in_channels: int,
    num_out_channels: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Randomly axis-align crop input and output image tensors.

  Args:
    input_img: Tensor with dimensions HWC.
    output_img: Tensor with dimensions HWC.
    sample_size: Side length (square) to crop to.
    num_in_channels: Number of channels in `input_img`.
    num_out_channels: Number of channels in `output_img`.
  Returns:
    input_img: Tensor with dimensions HWC.
    output_img: Tensor with dimensions HWC.
  """
  combined = tf.concat([input_img, output_img], axis=2)
  combined = tf.image.random_crop(
      combined,
      [sample_size, sample_size, num_in_channels + num_out_channels])
  input_img = combined[:, :, 0:num_in_channels]
  output_img = combined[:, :, -num_out_channels:]
  return input_img, output_img

# function to crop input and output images; from center of image
def center_crop_input_and_output_images(
    input_img: tf.Tensor,
    output_img: tf.Tensor,
    sample_size: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Calls `tf.image.central_crop` on input and output image tensors.

  Args:
    input_img: Tensor with dimensions HWC.
    output_img: Tensor with dimensions HWC.
    sample_size: Side length (square) to crop to.
  Returns:
    input_img: Tensor with dimensions HWC.
    output_img: Tensor with dimensions HWC.
  """
  central_fraction = sample_size / input_img.shape[0]
  input_img = tf.image.central_crop(input_img, central_fraction)
  output_img = tf.image.central_crop(output_img, central_fraction)
  return input_img, output_img
"""Dataset reader for Earth Engine data."""

# extract data keys
def _get_base_key(key: Text) -> Text:
  """Extracts the base key from the provided key.

  Earth Engine exports TFRecords containing each data variable with its
  corresponding variable name. In the case of time sequences, the name of the
  data variable is of the form 'variable_1', 'variable_2', ..., 'variable_n',
  where 'variable' is the name of the variable, and n the number of elements
  in the time sequence. Extracting the base key ensures that each step of the
  time sequence goes through the same normalization steps.
  The base key obeys the following naming pattern: '[a-zA-Z]+'
  For instance, for an input key 'variable_1', this function returns 'variable'.
  For an input key 'variable', this function simply returns 'variable'.

  Args:
    key: Input key.

  Returns:
    The corresponding base key.

  Raises:
    ValueError when `key` does not match the expected pattern.
  """
  match = re.match(r'[a-zA-Z]+', key)
  if match:
    return match.group(0)
  raise ValueError(
      f'The provided key does not match the expected pattern: {key}')

# function to clip and rescale image data values based on statistics
def _clip_and_rescale(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  """Clips and rescales inputs with the stats corresponding to `key`.

  Args:
    inputs: Inputs to clip and rescale.
    key: Key describing the inputs.

  Returns:
    Clipped and rescaled input.

  Raises:
    ValueError if there are no data statistics available for `key`.
  """
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, _, _ = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  return tf.math.divide_no_nan((inputs - min_val), (max_val - min_val))

# function to clip and normalize image data values based on statistics
def _clip_and_normalize(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  """Clips and normalizes inputs with the stats corresponding to `key`.

  Args:
    inputs: Inputs to clip and normalize.
    key: Key describing the inputs.

  Returns:
    Clipped and normalized input.

  Raises:
    ValueError if there are no data statistics available for `key`.
  """
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, mean, std = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  inputs = inputs - mean
  return tf.math.divide_no_nan(inputs, std)

# define data feature dictionary
def _get_features_dict(
    sample_size: int,
    features: List[Text],
) -> Dict[Text, tf.io.FixedLenFeature]:
  """Creates a features dictionary for TensorFlow IO.

  Args:
    sample_size: Size of the input tiles (square).
    features: List of feature names.

  Returns:
    A features dictionary for TensorFlow IO.
  """
  sample_shape = [sample_size, sample_size]
  features = set(features)
  columns = [
      tf.io.FixedLenFeature(shape=sample_shape, dtype=tf.float32)
      for _ in features
  ]
  return dict(zip(features, columns))

# function to read and format data as defined
def _parse_fn(
    example_proto: tf.train.Example, data_size: int, sample_size: int,
    num_in_channels: int, clip_and_normalize: bool,
    clip_and_rescale: bool, random_crop: bool, center_crop: bool,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Reads a serialized example.

  Args:
    example_proto: A TensorFlow example protobuf.
    data_size: Size of tiles (square) as read from input files.
    sample_size: Size the tiles (square) when input into the model.
    num_in_channels: Number of input channels.
    clip_and_normalize: True if the data should be clipped and normalized.
    clip_and_rescale: True if the data should be clipped and rescaled.
    random_crop: True if the data should be randomly cropped.
    center_crop: True if the data should be cropped in the center.

  Returns:
    (input_img, output_img) tuple of inputs and outputs to the ML model.
  """
  if (random_crop and center_crop):
    raise ValueError('Cannot have both random_crop and center_crop be True')
  input_features, output_features = INPUT_FEATURES, OUTPUT_FEATURES
  feature_names = input_features + output_features
  features_dict = _get_features_dict(data_size, feature_names)
  features = tf.io.parse_single_example(example_proto, features_dict)

  if clip_and_normalize:
    inputs_list = [
        _clip_and_normalize(features.get(key), key) for key in input_features
    ]
  elif clip_and_rescale:
    inputs_list = [
        _clip_and_rescale(features.get(key), key) for key in input_features
    ]
  else:
    inputs_list = [features.get(key) for key in input_features]

  inputs_stacked = tf.stack(inputs_list, axis=0)
  input_img = tf.transpose(inputs_stacked, [1, 2, 0])

  outputs_list = [features.get(key) for key in output_features]
  assert outputs_list, 'outputs_list should not be empty'
  outputs_stacked = tf.stack(outputs_list, axis=0)

  outputs_stacked_shape = outputs_stacked.get_shape().as_list()
  assert len(outputs_stacked.shape) == 3, ('outputs_stacked should be rank 3'
                                            'but dimensions of outputs_stacked'
                                            f' are {outputs_stacked_shape}')
  output_img = tf.transpose(outputs_stacked, [1, 2, 0])

  if random_crop:
    input_img, output_img = random_crop_input_and_output_images(
        input_img, output_img, sample_size, num_in_channels, 1)
  if center_crop:
    input_img, output_img = center_crop_input_and_output_images(
        input_img, output_img, sample_size)
  return input_img, output_img

# consolidated function to extract tfrecord data
def get_dataset(file_pattern: Text, data_size: int, sample_size: int,
                batch_size: int, num_in_channels: int, compression_type: Text,
                clip_and_normalize: bool, clip_and_rescale: bool,
                random_crop: bool, center_crop: bool) -> tf.data.Dataset:
  """Gets the dataset from the file pattern.

  Args:
    file_pattern: Input file pattern.
    data_size: Size of tiles (square) as read from input files.
    sample_size: Size the tiles (square) when input into the model.
    batch_size: Batch size.
    num_in_channels: Number of input channels.
    compression_type: Type of compression used for the input files.
    clip_and_normalize: True if the data should be clipped and normalized, False
      otherwise.
    clip_and_rescale: True if the data should be clipped and rescaled, False
      otherwise.
    random_crop: True if the data should be randomly cropped.
    center_crop: True if the data shoulde be cropped in the center.

  Returns:
    A TensorFlow dataset loaded from the input file pattern, with features
    described in the constants, and with the shapes determined from the input
    parameters to this function.
  """
  if (clip_and_normalize and clip_and_rescale):
    raise ValueError('Cannot have both normalize and rescale.')
  dataset = tf.data.Dataset.list_files(file_pattern)
  dataset = dataset.interleave(
      lambda x: tf.data.TFRecordDataset(x, compression_type=compression_type),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(
      lambda x: _parse_fn(  # pylint: disable=g-long-lambda
          x, data_size, sample_size, num_in_channels, clip_and_normalize,
          clip_and_rescale, random_crop, center_crop),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

## Extract Data

In [ ]:
# define filename pattern to identify train, eval, test files
file_pattern = 'train*'
file_pattern_eval = 'eval*'
file_pattern_test = 'test*'
print(file_pattern)
print(file_pattern_eval)
print(file_pattern_test)

In [20]:
# specify folder containing data downloaded from Google storage bucket
downloaddir = "/content/drive/My Drive/Colab Notebooks/Dissertation/data/folder_name"
os.chdir(downloaddir)

In [ ]:
# list files in download folder
! ls

In [25]:
# call function to extract data from files in download folder
# data_size = size of tile extracted from tfrecord
# sample_size = size of tile for model training
# batch_size = training data sample size to extract
# num_in_channels = number of training features to extract
# select EITHER clip_and_normalize=True OR clip_and_rescale=True
# select EITHER random_crop=False OR center_crop=True

# training dataset
dataset = get_dataset(
      file_pattern,
      data_size=128,
      sample_size=32,
      batch_size=2000,
      num_in_channels=18,
      compression_type="GZIP",
      clip_and_normalize=False,
      clip_and_rescale=True,
      random_crop=False,
      center_crop=True)

# evaluation dataset
dataset_eval = get_dataset(
      file_pattern_eval,
      data_size=128,
      sample_size=32,
      batch_size=2000,
      num_in_channels=18,
      compression_type="GZIP",
      clip_and_normalize=False,
      clip_and_rescale=True,
      random_crop=False,
      center_crop=True)

# testing dataset
dataset_test = get_dataset(
      file_pattern_test,
      data_size=128,
      sample_size=32,
      batch_size=400,
      num_in_channels=18,
      compression_type="GZIP",
      clip_and_normalize=False,
      clip_and_rescale=True,
      random_crop=False,
      center_crop=True)

## Extract Inputs and Labels

### Extract inputs and labels from extracted data

In [26]:
inputs, labels = next(iter(dataset))

In [27]:
inputs_eval, labels_eval = next(iter(dataset_eval))

In [28]:
inputs_test, labels_test = next(iter(dataset_test))

### Extract statistics for extracted data features

In [ ]:
# extract summary of values per band in tensors
tsz = inputs.shape[1]*inputs.shape[2]
# calculate dataset stats
for varx in range(0,inputs.shape[-1]):
  mi = None
  mx = None
  mn = None
  sd = None
  for x in range(0,inputs.shape[0]):
    if mi == None:
      mi = min(np.unique(inputs[x, :, :, varx]))
      mx = max(np.unique(inputs[x, :, :, varx]))
      mn = np.mean(inputs[x, :, :, varx])
      sd = np.std(inputs[x, :, :, varx])**2
    else:
      if (min(np.unique(inputs[x, :, :, varx])) < mi):
        mi = min(np.unique(inputs[x, :, :, varx]))
      if (max(np.unique(inputs[x, :, :, varx])) > mx):
        mx = max(np.unique(inputs[x, :, :, varx]))
      mn = mn + np.mean(inputs[x, :, :, varx])
      sd = sd + (np.std(inputs[x, :, :, varx])**2)
  print(f"Min {varx} - {INPUT_FEATURES[varx]}: {mi}")
  print(f"Max {varx} - {INPUT_FEATURES[varx]}: {mx}")
  print(f"Mean {varx} - {INPUT_FEATURES[varx]}: {mn/inputs.shape[0]}")
  print(f"Std {varx} - {INPUT_FEATURES[varx]}: {np.sqrt(sd/inputs.shape[0])}")

## Mapping Sample Inputs

In [30]:
# define feature names and plot parameters
# adapted from Huot et al. (2022)
TITLES = [
  'Elevation',
  'Population\ndensity',
  'Urban',
  'Drought',
  'Precip',
  'Energy\nrelease\ncomponent',
  'Humidity',
  'Wind\ndirection',
  'Wind\nvelocity',
  'Wind\ngust',
  'Wind\nU',
  'Wind\nV',
  'Temp',
  'NDVI',
  'NDII',
  'NBR',
  'dNBR',
  'Previous\nfire\nmask',
  'Fire\nmask'
]

n_rows = 4
n_features = inputs.shape[3]
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)

In [ ]:
# plot random sample of images
# adapted from Huot et al. (2022)
fig = plt.figure(figsize=(20,8))

nrows = random.sample(range(1, 250), n_rows)

for i in range(n_rows):
  k = nrows[i]
  for j in range(n_features + 1):
    plt.subplot(n_rows, n_features + 1, i * (n_features + 1) + j + 1)
    if i == 0:
      plt.title(TITLES[j], fontsize=13)
    if j < n_features - 1:
      plt.imshow(inputs[k, :, :, j], cmap='viridis')
    if j == n_features - 1:
      plt.imshow(inputs[k, :, :, -1], cmap=CMAP, norm=NORM)
    if j == n_features:
      plt.imshow(labels[k, :, :, 0], cmap=CMAP, norm=NORM) 
    plt.axis('off')
plt.tight_layout()

## U-Net model setup

### Define base component model blocks

In [34]:
# define double convolutional block
def double_conv2d_block(x, n_filters):
  # Conv2D 
  x = tf.keras.layers.Conv2D(n_filters, kernel_size=(3,3), strides=(1,1), padding = "same", kernel_initializer = "he_normal")(x)
  # Conv2D 
  x = tf.keras.layers.Conv2D(n_filters, kernel_size=(3,3), strides=(1,1), padding = "same", kernel_initializer = "he_normal")(x)
  return x

# define downsampling block
# includes double conv2d block, maxpool and dropout layers
def downsample_block(x, n_filters):
  # Conv2D 
  f = double_conv2d_block(x, n_filters)
  # downsample
  p = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(f)
  # dropout
  p = tf.keras.layers.Dropout(0.3)(p)
  return f, p

# define upsampling block
# includes transpose, concatenation and dropout layers followed by double conv2d block
def upsample_block(x, conv_features, n_filters):
  # upsample
  x = tf.keras.layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
  # concatenate
  x = tf.keras.layers.concatenate([x, conv_features])
  # dropout
  x = tf.keras.layers.Dropout(0.3)(x)
  # Conv2D 
  x = double_conv2d_block(x, n_filters)
  return x

### Define 2 layer U-Net

In [36]:
# define 2 layer U-NET model
def build_unet2_model(nfilters=32):

  input_shape = inputs.shape

  inp = tf.keras.layers.Input(shape=input_shape[1:])
  
  # downsample
  d1, do1 = downsample_block(inp, nfilters)
  # downsample
  d2, do2 = downsample_block(do1, (nfilters*2))
  # bottleneck
  bottleneck = double_conv2d_block(do2, (nfilters*4))
  # upsample
  u1 = upsample_block(bottleneck, d2, (nfilters*2))
  # upsample
  u2 = upsample_block(u1, d1, nfilters)

  # outputs
  outputs = tf.keras.layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u2)

  # unet model with Keras Functional API
  unet_model = tf.keras.Model(inp, outputs, name="U-Net")
  
  return unet_model

### Define 3 layer U-Net

In [37]:
# define 3 layer U-NET model
def build_unet3_model(nfilters=32):

  input_shape = inputs.shape

  inp = tf.keras.layers.Input(shape=input_shape[1:])
  
  # downsample
  d1, do1 = downsample_block(inp, nfilters)
  # downsample
  d2, do2 = downsample_block(do1, (nfilters*2))
  # downsample
  d3, do3 = downsample_block(do2, (nfilters*4))
  # bottleneck
  bottleneck = double_conv2d_block(do3, (nfilters*8))
  # upsample
  u1 = upsample_block(bottleneck, d3, (nfilters*4))
  # upsample
  u2 = upsample_block(u1, d2, (nfilters*2))
  # upsample
  u3 = upsample_block(u2, d1, nfilters)

  # outputs
  outputs = tf.keras.layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u3)

  # unet model with Keras Functional API
  unet_model = tf.keras.Model(inp, outputs, name="U-Net")
  
  return unet_model

### Define 4 layer U-Net

In [38]:
# define 4 layer U-NET model
def build_unet4_model(nfilters=32):

  input_shape = inputs.shape

  inp = tf.keras.layers.Input(shape=input_shape[1:])
  
  # downsample
  d1, do1 = downsample_block(inp, nfilters)
  # downsample
  d2, do2 = downsample_block(do1, (nfilters*2))
  # downsample
  d3, do3 = downsample_block(do2, (nfilters*4))
  # downsample
  d4, do4 = downsample_block(do3, (nfilters*8))
  # bottleneck
  bottleneck = double_conv2d_block(do4, (nfilters*16))
  # upsample
  u1 = upsample_block(bottleneck, d4, (nfilters*8))
  # upsample
  u2 = upsample_block(u1, d3, (nfilters*4))
  # upsample
  u3 = upsample_block(u2, d2, (nfilters*2))
  # upsample
  u4 = upsample_block(u3, d1, nfilters)

  # outputs
  outputs = tf.keras.layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u4)

  # unet model with Keras Functional API
  unet_model = tf.keras.Model(inp, outputs, name="U-Net")
  
  return unet_model

### Define custom metrics to evaluate model performance

The following code cell was sourced from the following link:

https://gist.github.com/arnaldog12/5f2728f229a8bd3b4673b72786913252

In [33]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

## Compile and train model

In [53]:
# define model based on required number of layers and filters
unet_model = build_unet3_model(nfilters=32)

In [ ]:
# print model summary
unet_model.summary()

In [ ]:
# plot model summary
tf.keras.utils.plot_model(unet_model, rankdir='LR', show_shapes=False, show_layer_activations=False)

In [54]:
# define optimisation function and learning rate
opt = keras.optimizers.Adam(learning_rate=0.001)

In [55]:
# compile model including optimizer, loss function and required metrics
unet_model.compile(optimizer=opt,
                  loss="binary_crossentropy",
                  metrics=["accuracy",
                           tf.keras.metrics.AUC(name='auc'),
                           f1_m,
                           precision_m,
                           recall_m,
                           tf.keras.metrics.BinaryIoU(target_class_ids=[1],threshold=0.5,name='iou')])

In [ ]:
# train model for defined number of epochs
# include in training validation split of 20%
model_history = unet_model.fit(inputs, labels,
                               epochs=40,
                               validation_split=0.2)

## Evaluate trained model and plot metrics

In [64]:
# save trained model to working folder for future evaluation
save_model_as = "model_name"
unet_model.save(save_model_as+".h5",save_format='h5')

In [ ]:
# extract metrics values for the trained network using the evaluation dataset
evalscores = unet_model.evaluate(inputs_eval, labels_eval, verbose=0)

print(f"Evaluation Loss: {evalscores[0]:.05f}")
print(f"Evaluation Accuracy: {evalscores[1]:.05f}")
print(f"Evaluation AUC: {evalscores[2]:.05f}")
print(f"Evaluation F1: {evalscores[3]:.05f}")
print(f"Evaluation Precision: {evalscores[4]:.05f}")
print(f"Evaluation Recall: {evalscores[5]:.05f}")
print(f"Evaluation IoU: {evalscores[6]:.05f}")

In [60]:
# define eval metrics df to save evaluated metrics
column_names = ["Model", "Dataset", "Loss", "Accuracy",
                "AUC", "F1", "Precision", "Recall", "IoU"]

evalmetrics = pd.DataFrame(columns = column_names)

In [ ]:
# define model name to associate with results
model_settings = "model_name"
# write evaluation metrics to df
eval_new_row = {"Model": model_settings,
                "Dataset": downloaddir.split("/")[7],
                "Loss": evalscores[0], "Accuracy": evalscores[1],
                "AUC": evalscores[2], "F1": evalscores[3],
                "Precision": evalscores[4], "Recall": evalscores[5],
                "IoU": evalscores[6]}
evalmetrics = evalmetrics.append(eval_new_row, ignore_index=True)
# print df metrics
print(evalmetrics)

In [63]:
# export evalmetrics df to csv when all required analyses complete
test_csv_file = os.path.join(downloaddir,"output.csv")
with open(test_csv_file, mode='w') as f:
  evalmetrics.to_csv(f)

In [ ]:
# plot model training history
# define plot attributes
fig = plt.figure()
ax = plt.subplot(111)
ax.set_ylabel('Metric Value')
ax.set_xlabel('epoch')
ax.set_ylim([0.1, 1])
ax.grid(color='grey', linestyle='-.', linewidth=0.5)

# plot history
ax.plot(model_history.history['auc'],color='red',label='Train-AUC')
ax.plot(model_history.history['val_auc'],color='darkred',label='Eval-AUC')
ax.plot(model_history.history['f1_m'],color='yellow',label='Train-F1')
ax.plot(model_history.history['val_f1_m'],color='darkorange',label='Eval-F1')
ax.plot(model_history.history['iou'],color='blue',label='Train-IoU')
ax.plot(model_history.history['val_iou'],color='darkblue',label='Eval-IoU')

# shrink axis by 20% to place legend outside plot extent
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# place legend to the right of the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [66]:
# write model training history to df
hist_df = pd.DataFrame(model_history.history) 

In [67]:
# saved model history to csv for future analysis
hist_csv_file = os.path.join(downloaddir,save_model_as+".csv")
with open(hist_csv_file, mode='w') as f:
  hist_df.to_csv(f)

## Load saved model for testing against loaded dataset

In [68]:
# define folder name for saved model and model name
savedmodeldir = "/content/drive/My Drive/Colab Notebooks/Dissertation/data/folder_name"
os.chdir(savedmodeldir)
# load saved model from source directory
model_to_load = "model_name.h5"
unet_model_open = keras.models.load_model(model_to_load,
                                          custom_objects={"f1_m": f1_m,
                                                          "recall_m": recall_m,
                                                          "precision_m": precision_m})

In [69]:
# define eval metrics df to save evaluated metrics
column_names = ["Model", "Dataset", "Loss", "Accuracy",
                "AUC", "F1", "Precision", "Recall", "IoU"]

evalmetrics = pd.DataFrame(columns = column_names)

In [ ]:
# check summary of loaded model
unet_model_open.summary()

In [ ]:
# evaluate loaded model based on current dataset
evalscores = unet_model_open.evaluate(inputs_eval, labels_eval, verbose=0)

print(f"Evaluation Loss: {evalscores[0]:.05f}")
print(f"Evaluation Accuracy: {evalscores[1]:.05f}")
print(f"Evaluation AUC: {evalscores[2]:.05f}")
print(f"Evaluation F1: {evalscores[3]:.05f}")
print(f"Evaluation Precision: {evalscores[4]:.05f}")
print(f"Evaluation Recall: {evalscores[5]:.05f}")
print(f"Evaluation IoU: {evalscores[6]:.05f}")

In [ ]:
# write evaluation metrics to df
eval_new_row = {"Model": model_to_load,
                "Dataset": downloaddir.split("/")[7],
                "Loss": evalscores[0], "Accuracy": evalscores[1],
                "AUC": evalscores[2], "F1": evalscores[3],
                "Precision": evalscores[4], "Recall": evalscores[5],
                "IoU": evalscores[6]}
evalmetrics = evalmetrics.append(eval_new_row, ignore_index=True)
print(evalmetrics)

In [73]:
# export evalmetrics df to csv when all required analyses complete
pred_csv_file = os.path.join(downloaddir,"eval_output.csv")
with open(pred_csv_file, mode='w') as f:
  evalmetrics.to_csv(f)

## Predict wildfire spread from loaded model and plot

In [74]:
# define functions to assess difference between prediction and label

# calculate difference
def diff_func(pred, act):
  act_np = act.numpy().astype(np.int8)
  diff_t = np.where(pred != act_np, pred-act_np, pred+act_np)
  return diff_t

# calculate union
def add_func(pred, act):
  act_np = act.numpy().astype(np.int8)
  add_t = np.where(pred != act_np, pred+act_np, pred*act_np)
  return add_t

In [ ]:
# extract model prediction at varying probability levels
upred = unet_model_open.predict(inputs)
upredm25 = (upred >= 0.25).astype(np.int8)
upredm = (upred >= 0.5).astype(np.int8)
upredm75 = (upred >= 0.75).astype(np.int8)

In [76]:
# apply functions to predictions
# 50% probability
diff_ = diff_func(upredm,labels)
add_ = add_func(upredm,labels)
# 25% probability
diff_25 = diff_func(upredm25,labels)
add_25 = add_func(upredm25,labels)
# 75% probability
diff_75 = diff_func(upredm75,labels)
add_75 = add_func(upredm75,labels)

In [77]:
# summarise prediction statistics

# define output df - 50%
output_pred = pd.DataFrame(columns=['row','missed','wrong','correct','score','label'])
# write output to df
for x in range(0,diff_.shape[0]):
  orig = np.sum(labels[x, :, :, 0])
  missed = (sum(diff_[x, :, :, 0][diff_[x, :, :, 0]==-1])*-1)
  wrong = sum(diff_[x, :, :, 0][diff_[x, :, :, 0]==1])
  correct = (sum(diff_[x, :, :, 0][diff_[x, :, :, 0]==2])/2)
  score = (correct-(wrong+missed))
  output_pred.loc[len(output_pred.index)] = [x, missed, wrong, correct, score, orig]

# repeat for 25%
output_pred_25 = pd.DataFrame(columns=['row','missed','wrong','correct','score','label'])
# write output to df
for x in range(0,diff_25.shape[0]):
  orig = np.sum(labels[x, :, :, 0])
  missed = (sum(diff_25[x, :, :, 0][diff_25[x, :, :, 0]==-1])*-1)
  wrong = sum(diff_25[x, :, :, 0][diff_25[x, :, :, 0]==1])
  correct = (sum(diff_25[x, :, :, 0][diff_25[x, :, :, 0]==2])/2)
  score = (correct-(wrong+missed))
  output_pred_25.loc[len(output_pred_25.index)] = [x, missed, wrong, correct, score, orig]

# repeat for 75%
output_pred_75 = pd.DataFrame(columns=['row','missed','wrong','correct','score','label'])
# write output to df
for x in range(0,diff_75.shape[0]):
  orig = np.sum(labels[x, :, :, 0])
  missed = (sum(diff_75[x, :, :, 0][diff_75[x, :, :, 0]==-1])*-1)
  wrong = sum(diff_75[x, :, :, 0][diff_75[x, :, :, 0]==1])
  correct = (sum(diff_75[x, :, :, 0][diff_75[x, :, :, 0]==2])/2)
  score = (correct-(wrong+missed))
  output_pred_75.loc[len(output_pred_75.index)] = [x, missed, wrong, correct, score, orig]

# write output to csv
output_pred.to_csv(os.path.join(savedmodeldir,model_to_load.split(".")[0]+"-runonmodel.csv"))

In [78]:
# define sample arrays per data quartile
arrayq1 = list(range(1, 500))
arrayq2 = list(range(501, 1000))
arrayq3 = list(range(1001, 1500))
arrayq4 = list(range(1501, 2000))

In [ ]:
# extract best/worst match per sample quartile

# 50%
topfour = [output_pred[output_pred['row'].isin(arrayq1)]['score'].nlargest(n=1).index[0]]
topfour.append(output_pred[output_pred['row'].isin(arrayq2)]['score'].nlargest(n=1).index[0])
topfour.append(output_pred[output_pred['row'].isin(arrayq3)]['score'].nlargest(n=1).index[0])
topfour.append(output_pred[output_pred['row'].isin(arrayq4)]['score'].nlargest(n=1).index[0])
botfour = [output_pred[output_pred['row'].isin(arrayq1)]['score'].nsmallest(n=1).index[0]]
botfour.append(output_pred[output_pred['row'].isin(arrayq2)]['score'].nsmallest(n=1).index[0])
botfour.append(output_pred[output_pred['row'].isin(arrayq3)]['score'].nsmallest(n=1).index[0])
botfour.append(output_pred[output_pred['row'].isin(arrayq4)]['score'].nsmallest(n=1).index[0])

# 25%
topfour25 = [output_pred_25[output_pred_25['row'].isin(arrayq1)]['score'].nlargest(n=1).index[0]]
topfour25.append(output_pred_25[output_pred_25['row'].isin(arrayq2)]['score'].nlargest(n=1).index[0])
topfour25.append(output_pred_25[output_pred_25['row'].isin(arrayq3)]['score'].nlargest(n=1).index[0])
topfour25.append(output_pred_25[output_pred_25['row'].isin(arrayq4)]['score'].nlargest(n=1).index[0])
botfour25 = [output_pred_25[output_pred_25['row'].isin(arrayq1)]['score'].nsmallest(n=1).index[0]]
botfour25.append(output_pred_25[output_pred_25['row'].isin(arrayq2)]['score'].nsmallest(n=1).index[0])
botfour25.append(output_pred_25[output_pred_25['row'].isin(arrayq3)]['score'].nsmallest(n=1).index[0])
botfour25.append(output_pred_25[output_pred_25['row'].isin(arrayq4)]['score'].nsmallest(n=1).index[0])

# 75%
topfour75 = [output_pred_75[output_pred_75['row'].isin(arrayq1)]['score'].nlargest(n=1).index[0]]
topfour75.append(output_pred_75[output_pred_75['row'].isin(arrayq2)]['score'].nlargest(n=1).index[0])
topfour75.append(output_pred_75[output_pred_75['row'].isin(arrayq3)]['score'].nlargest(n=1).index[0])
topfour75.append(output_pred_75[output_pred_75['row'].isin(arrayq4)]['score'].nlargest(n=1).index[0])
botfour75 = [output_pred_75[output_pred_75['row'].isin(arrayq1)]['score'].nsmallest(n=1).index[0]]
botfour75.append(output_pred_75[output_pred_75['row'].isin(arrayq2)]['score'].nsmallest(n=1).index[0])
botfour75.append(output_pred_75[output_pred_75['row'].isin(arrayq3)]['score'].nsmallest(n=1).index[0])
botfour75.append(output_pred_75[output_pred_75['row'].isin(arrayq4)]['score'].nsmallest(n=1).index[0])

# print identified top/bottom four sampled images
print(topfour)
print(botfour)

### Plot sample of top/bottom four predictions

In [80]:
# define datasets from above for plotting
predtomap = upredm
difftomap = diff_
tomaptop = topfour
tomapbot = botfour

In [ ]:
# Show Top 4 Predictions
# define tile names
TITLES_DIFF = [
  'Previous Day Fire', 'Next Day Fire',
  'Predicted\nNext Day Fire', 'Prediction vs Next Day'
]

# define plot attributes and parameters
fig = plt.figure(figsize=(16,12))
n_rows = len(tomaptop)
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
diff_classes = [-1,-0.01,0.99,1.99,2]
diff_colors = ['black', 'silver', 'yellow', 'orangered']
diff_cmap = ListedColormap(diff_colors)
diff_norm = BoundaryNorm(diff_classes, len(diff_colors))

for i in range(n_rows):
  k = tomaptop[i]
  print(k)
  for j in range(4):
    plt.subplot(n_rows, 4, i * (4) + j + 1)
    if i == 0:
      plt.title(TITLES_DIFF[j], fontsize=14)
    if j == 0:
      plt.imshow(inputs[k, :, :, -1], cmap=CMAP, norm=NORM)
    if j == 1:
      plt.imshow(labels[k, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == 2:
      plt.imshow(predtomap[k, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == 3:
      plt.imshow(difftomap[k, :, :, 0], cmap=diff_cmap, norm=diff_norm) 
    plt.axis('off')
plt.tight_layout()

In [ ]:
# Show Bottom 4 Predictions
# define tile names
TITLES_DIFF = [
  'Previous Day Fire', 'Next Day Fire',
  'Predicted\nNext Day Fire', 'Prediction vs Next Day'
]

# define plot attributes and parameters
fig = plt.figure(figsize=(16,12))
n_rows = len(tomapbot)
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
diff_classes = [-1,-0.01,0.99,1.99,2]
diff_colors = ['black', 'silver', 'yellow', 'orangered']
diff_cmap = ListedColormap(diff_colors)
diff_norm = BoundaryNorm(diff_classes, len(diff_colors))

for i in range(n_rows):
  k = tomapbot[i]
  print(k)
  for j in range(4):
    plt.subplot(n_rows, 4, i * (4) + j + 1)
    if i == 0:
      plt.title(TITLES_DIFF[j], fontsize=14)
    if j == 0:
      plt.imshow(inputs[k, :, :, -1], cmap=CMAP, norm=NORM)
    if j == 1:
      plt.imshow(labels[k, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == 2:
      plt.imshow(predtomap[k, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == 3:
      plt.imshow(difftomap[k, :, :, 0], cmap=diff_cmap, norm=diff_norm) 
    plt.axis('off')
plt.tight_layout()

### Plot sample of predictions with underlying data

In [83]:
# Plot all layers with predictions
# define tile names
TITLES = ['Elevation', 'Population\ndensity', 'Urban',
          'Drought', 'Precip', 'Energy\nrelease\ncomponent', 'Humidity',
          'Wind\ndirection', 'Wind\nvelocity', 'Wind\nGust', 'Wind\nU', 'Wind\nV',
          'Temp', 'NDVI', 'NDII', 'NBR', 'dNBR',
          'Previous\nfire\nmask', 'Fire\nmask', 'Fire\pred', 'Diff']

# define plot attributes and parameters
addn = 0
n_features = inputs.shape[3]
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
CMAP_DIFF = colors.ListedColormap(['black', 'silver', 'yellow', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
BOUNDS_DIFF = [-1, -0.01, 0.01, 0.99, 1.01, 1.99,  2]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
NORM_DIFF = colors.BoundaryNorm(BOUNDS_DIFF, CMAP_DIFF.N)
NORM_V = colors.NoNorm(vmin=0, vmax=1)
diff_classes = [-1,-0.01,0.99,1.99,2]
diff_colors = ['black', 'silver', 'yellow', 'orangered']
diff_cmap = ListedColormap(diff_colors)
diff_norm = BoundaryNorm(diff_classes, len(diff_colors))

In [ ]:
# plot random sample for top four
fig = plt.figure(figsize=(24,8))

n_rows = len(tomaptop)

for i in range(n_rows):
  k = tomaptop[i]
  for j in range(n_features + 3):
    plt.subplot(n_rows, n_features + 3, i * (n_features + 3) + j + 1)
    if i == 0:
      plt.title(TITLES[j], fontsize=13)
    if j < n_features - 1:
      plt.imshow(inputs[k+addn, :, :, j], cmap='viridis')
    if j == n_features - 1:
      plt.imshow(inputs[k+addn, :, :, -1], cmap=CMAP, norm=NORM)
    if j == n_features:
      plt.imshow(labels[k+addn, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == n_features + 1:
      plt.imshow(upredm[k+addn, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == n_features + 2:
      plt.imshow(diff_[k+addn, :, :, 0], cmap=diff_cmap, norm=diff_norm) 
    plt.axis('off')
plt.tight_layout()

In [ ]:
# plot random sample for bottom four
fig = plt.figure(figsize=(24,8))

n_rows = len(tomapbot)

for i in range(n_rows):
  k = tomapbot[i]
  for j in range(n_features + 3):
    plt.subplot(n_rows, n_features + 3, i * (n_features + 3) + j + 1)
    if i == 0:
      plt.title(TITLES[j], fontsize=13)
    if j < n_features - 1:
      plt.imshow(inputs[k+addn, :, :, j], cmap='viridis')
    if j == n_features - 1:
      plt.imshow(inputs[k+addn, :, :, -1], cmap=CMAP, norm=NORM)
    if j == n_features:
      plt.imshow(labels[k+addn, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == n_features + 1:
      plt.imshow(upredm[k+addn, :, :, 0], cmap=CMAP, norm=NORM) 
    if j == n_features + 2:
      plt.imshow(diff_[k+addn, :, :, 0], cmap=diff_cmap, norm=diff_norm) 
    plt.axis('off')
plt.tight_layout()